In [2]:
import sys
import hail as hl
import pandas as pd
import gnomad
import gnomad.utils.vep
from gnomad.resources.grch38 import gnomad
from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot
import pprint as pp
import numpy as np

In [3]:
print("load QCed mt...")
mt = hl.read_matrix_table("gs://pbn_pilot/proj-PBN/04_rare_SNPs/00_raw_data/03_filter_entries_VarQC.SCZ.mt")

print("load vep ht...")
variants = hl.read_table("gs://pbn_pilot/proj-PBN/04_rare_SNPs/00_raw_data/02_SCZ_BGEcallset.varQCed.min_rep.VEP.ht")   

Initializing Hail with default parameters...


load QCed mt...


/opt/conda/miniconda3/lib/python3.11/site-packages/hailtop/aiocloud/aiogoogle/user_config.py:43: UserWarning:

Reading spark-defaults.conf to determine GCS requester pays configuration. This is deprecated. Please use `hailctl config set gcs_requester_pays/project` and `hailctl config set gcs_requester_pays/buckets`.

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


SPARKMONITOR_LISTENER: Started SparkListener for Jupyter Notebook
SPARKMONITOR_LISTENER: Port obtained from environment: 41989
SPARKMONITOR_LISTENER: Application Started: application_1752686110983_0001 ...Start Time: 1752686805012


Running on Apache Spark version 3.5.0
SparkUI available at http://ychen-cnv-m.us-central1-a.c.pbn-analysis.internal:41657
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.135-034ef3e08116
LOGGING: writing to /home/hail/hail-20250716-1726-0.2.135-034ef3e08116.log


load vep ht...


In [4]:
print("Annotating mt with VEP annotations...")
mt = mt.annotate_rows(vep = variants[mt.row_key].vep)
mt = mt.filter_rows(hl.is_defined(mt.vep))

print(list(mt.vep)) # no 'worst_csq_by_gene_canonical' info here

Annotating mt with VEP annotations...
['assembly_name', 'allele_string', 'ancestral', 'colocated_variants', 'context', 'end', 'id', 'input', 'intergenic_consequences', 'most_severe_consequence', 'motif_feature_consequences', 'regulatory_feature_consequences', 'seq_region_name', 'start', 'strand', 'transcript_consequences', 'variant_class']


In [5]:
print("make LOF HC loftee ")
from gnomad.utils.vep import process_consequences
mt=process_consequences(mt)

make LOF HC loftee 


In [6]:
print(list(mt.vep)) # generate 'worst_csq_by_gene_canonical' info after process consequences

['assembly_name', 'allele_string', 'ancestral', 'colocated_variants', 'context', 'end', 'id', 'input', 'intergenic_consequences', 'most_severe_consequence', 'motif_feature_consequences', 'regulatory_feature_consequences', 'seq_region_name', 'start', 'strand', 'transcript_consequences', 'variant_class', 'worst_consequence_term', 'worst_csq_by_gene', 'worst_csq_for_variant', 'worst_csq_by_gene_canonical', 'worst_csq_for_variant_canonical']


In [7]:
mt.vep.worst_csq_by_gene_canonical.consequence_terms.show() # before explode

,,
locus,alleles,<expr>
locus<GRCh38>,array<str>,array<array<str>>
chr1:930204,"[""G"",""A""]","[[""missense_variant""]]"
chr1:930245,"[""G"",""A""]","[[""missense_variant""]]"
chr1:930285,"[""G"",""A""]","[[""missense_variant""]]"
chr1:930314,"[""C"",""T""]","[[""missense_variant""]]"
chr1:930320,"[""C"",""T""]","[[""missense_variant""]]"
chr1:935835,"[""C"",""G""]","[[""synonymous_variant""]]"
chr1:935836,"[""G"",""A""]","[[""missense_variant""]]"
chr1:939121,"[""C"",""T""]","[[""missense_variant""]]"


In [8]:
mt = mt.explode_rows(mt.vep.worst_csq_by_gene_canonical)

In [9]:
print("2. explode by worst_csq_by_gene_canonical ---> Sites: %d\n" % (mt.count_rows())) # 5495541
print(list(mt.vep.worst_csq_by_gene_canonical)) # check 'lof'
print("Check how many LOF HC variants")
lof_hc = mt.filter_rows(mt.vep.worst_csq_by_gene_canonical.lof.contains('HC')) # Konrad

2. explode by worst_csq_by_gene_canonical ---> Sites: 227896

['allele_num', 'amino_acids', 'appris', 'biotype', 'canonical', 'ccds', 'cdna_start', 'cdna_end', 'cds_end', 'cds_start', 'codons', 'consequence_terms', 'distance', 'domains', 'exon', 'gene_id', 'gene_pheno', 'gene_symbol', 'gene_symbol_source', 'hgnc_id', 'hgvsc', 'hgvsp', 'hgvs_offset', 'impact', 'intron', 'lof', 'lof_flags', 'lof_filter', 'lof_info', 'minimised', 'polyphen_prediction', 'polyphen_score', 'protein_end', 'protein_start', 'protein_id', 'sift_prediction', 'sift_score', 'strand', 'swissprot', 'transcript_id', 'trembl', 'tsl', 'uniparc', 'variant_allele', 'most_severe_consequence', 'csq_score']
Check how many LOF HC variants


In [10]:
print("3. HC loftee from worst_csq_by_gene_canonical ---> Sites: %d\n" % (lof_hc.count_rows())) # 41073
print("4. File saving...")
mt = mt.checkpoint("gs://pbn_pilot/proj-PBN/04_rare_SNPs/00_raw_data/04_explode_by_worst_csq_by_gene_canonical.mt")

3. HC loftee from worst_csq_by_gene_canonical ---> Sites: 1556

4. File saving...


FatalError: HailException: RelationalSetup.writeMetadata: file already exists: gs://pbn_pilot/proj-PBN/04_rare_SNPs/00_raw_data/04_explode_by_worst_csq_by_gene_canonical.mt

Java stack trace:
is.hail.utils.HailException: RelationalSetup.writeMetadata: file already exists: gs://pbn_pilot/proj-PBN/04_rare_SNPs/00_raw_data/04_explode_by_worst_csq_by_gene_canonical.mt
	at __C4819Compile.__m4823begin_group_0(Emit.scala)
	at __C4819Compile.__m4821split_Block(Emit.scala)
	at __C4819Compile.apply(Emit.scala)
	at is.hail.expr.ir.CompileAndEvaluate$.$anonfun$_apply$3(CompileAndEvaluate.scala:64)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at is.hail.utils.ExecutionTimer.time(ExecutionTimer.scala:98)
	at is.hail.backend.ExecuteContext.time(ExecuteContext.scala:187)
	at is.hail.expr.ir.CompileAndEvaluate$.$anonfun$_apply$2(CompileAndEvaluate.scala:64)
	at is.hail.expr.ir.CompileAndEvaluate$.$anonfun$_apply$2$adapted(CompileAndEvaluate.scala:62)
	at is.hail.backend.E

In [11]:
mt = mt.filter_rows(mt.vep.worst_csq_by_gene_canonical.biotype == "protein_coding")
mt.count() # (4178002, 8966)

(183100, 70)

In [12]:
mt.vep.worst_csq_by_gene_canonical.show()

+---------------+------------+-------------------+--------------------+
| locus         | alleles    | <expr>.allele_num | <expr>.amino_acids |
+---------------+------------+-------------------+--------------------+
| locus<GRCh38> | array<str> |             int32 | str                |
+---------------+------------+-------------------+--------------------+
| chr1:930204   | ["G","A"]  |                 1 | "R/Q"              |
| chr1:930245   | ["G","A"]  |                 1 | "D/N"              |
| chr1:930285   | ["G","A"]  |                 1 | "R/Q"              |
| chr1:930314   | ["C","T"]  |                 1 | "H/Y"              |
| chr1:930320   | ["C","T"]  |                 1 | "R/C"              |
| chr1:935835   | ["C","G"]  |                 1 | "P"                |
| chr1:935836   | ["G","A"]  |                 1 | "E/K"              |
| chr1:939121   | ["C","T"]  |                 1 | "S/L"              |
| chr1:939285   | ["G","A"]  |                 1 | "L"                |
| chr1:939285   | ["G","A"]  |                 1 | NA                 |
+---------------+------------+-------------------+--------------------+

+---------------+------------------+------------------+-------------+
| <expr>.appris | <expr>.biotype   | <expr>.canonical | <expr>.ccds |
+---------------+------------------+------------------+-------------+
| str           | str              |            int32 | str         |
+---------------+------------------+------------------+-------------+
| "P2"          | "protein_coding" |                1 | "CCDS2.2"   |
| "P2"          | "protein_coding" |                1 | "CCDS2.2"   |
| "P2"          | "protein_coding" |                1 | "CCDS2.2"   |
| "P2"          | "protein_coding" |                1 | "CCDS2.2"   |
| "P2"          | "protein_coding" |                1 | "CCDS2.2"   |
| "P2"          | "protein_coding" |                1 | "CCDS2.2"   |
| "P2"          | "protein_coding" |                1 | "CCDS2.2"   |
| "P2"          | "protein_coding" |                1 | "CCDS2.2"   |
| "P2"          | "protein_coding" |                1 | "CCDS2.2"   |
| "P1"          | "protein_coding" |                1 | "CCDS3.1"   |
+---------------+------------------+------------------+-------------+

+-------------------+-----------------+----------------+------------------+
| <expr>.cdna_start | <expr>.cdna_end | <expr>.cds_end | <expr>.cds_start |
+-------------------+-----------------+----------------+------------------+
|             int32 |           int32 |          int32 |            int32 |
+-------------------+-----------------+----------------+------------------+
|               205 |             205 |            122 |              122 |
|               246 |             246 |            163 |              163 |
|               286 |             286 |            203 |              203 |
|               315 |             315 |            232 |              232 |
|               321 |             321 |            238 |              238 |
|               452 |             452 |            369 |              369 |
|               453 |             453 |            370 |              370 |
|               595 |             595 |            512 |              512 |
|               614 |             614 |            531 |              531 |
|                NA |              NA |             NA |               NA |
+-------------------+-----------------+----------------+------------------+

+---------------+-----------------------------+-----------------+
| <expr>.codons | <expr>.consequence_terms    | <expr>.distance |
+---------------+-----------------------------+-----------------+
| str           | array<str>                  |           int32 |
+---------------+-----------------------------+-----------------+
| "cGg/cAg"     | ["missense_variant"]        |              NA |
| "Gat/Aat"     | ["missense_variant"]        |              NA |
| "cGg/cAg"     | ["missense_variant"]        |

In [13]:
def add_vep_annotations(mt):

    def prioritized(field, mt):
        vep = mt.vep.worst_csq_by_gene_canonical
        return hl.if_else(
            (vep.canonical == 1) & (vep.biotype == "protein_coding") & hl.is_defined(vep.amino_acids),
            getattr(vep, field),
            hl.if_else(
                (vep.canonical == 1) & (vep.biotype == "protein_coding"),
                getattr(vep, field),
                hl.if_else(
                    (vep.canonical == 1),
                    getattr(vep, field),
                    getattr(vep, field)
                )
            )
        )

    mt = mt.annotate_rows(
        gene = prioritized("gene_symbol", mt),
        gene_id = prioritized("gene_id", mt),
        transcript_id = prioritized("transcript_id", mt),
        hgnc_id = prioritized("hgnc_id", mt),
        hgvsc = prioritized("hgvsc", mt),
        hgvsp = prioritized("hgvsp", mt),
        consequence = prioritized("consequence_terms", mt),
        lof = prioritized("lof", mt),
        lof_flags = prioritized("lof_flags", mt),
        polyphen = prioritized("polyphen_prediction", mt),
        sift = prioritized("sift_prediction", mt)
    )

    mt = mt.annotate_rows(
        isPTV = mt.consequence.contains('frameshift_variant') |
                mt.consequence.contains('stop_gained') |
                mt.consequence.contains('splice_donor_variant') |
                mt.consequence.contains('splice_acceptor_variant') |
                mt.consequence.contains('transcript_ablation'),
        isMIS = mt.consequence.contains('missense_variant'),
        isSYN = mt.consequence.contains('synonymous_variant') |
                mt.consequence.contains('stop_retained_variant'),
        isSRG = mt.consequence.contains('splice_region_variant'),
        isSSL = mt.consequence.contains('start_lost') |
                mt.consequence.contains('stop_lost'),
        isINF = mt.consequence.contains('inframe_insertion') |
                mt.consequence.contains('inframe_deletion')
    )

    mt = mt.annotate_rows(
        SII = hl.literal([
            'frameshift_variant', 'stop_gained', 'splice_donor_variant',
            'splice_acceptor_variant', 'transcript_ablation', 'missense_variant',
            'synonymous_variant', 'stop_retained_variant', 'splice_region_variant',
            'start_lost', 'stop_lost', 'inframe_insertion', 'inframe_deletion']
        ).contains(mt.vep.worst_csq_by_gene_canonical.most_severe_consequence)
    )

    mt = mt.annotate_rows(
        isSNP = hl.is_snp(mt.alleles[0], mt.alleles[1]),
        isIndel = hl.is_indel(mt.alleles[0], mt.alleles[1])
    )

    return mt

In [14]:
# Add VEP annotations
mt = add_vep_annotations(mt)

In [15]:
mt.count()

(183100, 70)

In [16]:
mpc = hl.read_table("gs://pbn_pilot/proj-PBN/misc/mpc.LiftOverhg38.ht") # MPC scores
mt = mt.annotate_rows(mpc = mpc[mt.row_key]) 
mt = mt.annotate_rows(MPCscore = mt.mpc.MPC)
mt = mt.drop('mpc')

In [17]:
mt = hl.variant_qc(mt)

In [18]:
mt = mt.annotate_rows(
    mpc3 = mt.MPCscore > 3,
    mpc2_to_3 = (mt.MPCscore >= 2) & (mt.MPCscore <= 3),
    mpc_less_2 = mt.MPCscore < 2,
    mpc2 = mt.MPCscore >= 2,
       
    isSingleton = mt.variant_qc.n_non_ref == 1,
    AC5 = mt.variant_qc.AC[1] < 6,
    AC10 = mt.variant_qc.AC[1] < 11
)

In [19]:
# Filter based on annotations, keeping coding variants and variants in evaluation regions
mt = mt.filter_rows(mt.isPTV | mt.isMIS | mt.isSYN | mt.isSRG | mt.isSSL | mt.isINF | 
                    mt.SII, keep = True) 
mt.count() # (4539601, 44906)

(144695, 70)

In [20]:
# Need minimal representation of alleles
mt = mt.annotate_rows(min_rep = hl.min_rep(mt.locus, mt.alleles))
mt = mt.key_rows_by('min_rep')
mt = mt.drop('locus', 'alleles')

mt = mt.annotate_rows(locus = mt.min_rep.locus,
                      alleles = mt.min_rep.alleles)
mt = mt.key_rows_by('locus', 'alleles')
mt = mt.drop('min_rep')

In [21]:
# Write temp file
mt.checkpoint('gs://pbn_pilot/proj-PBN/04_rare_SNPs/00_raw_data/05_vep_explode_coding_annot.mt')

2025-07-16 17:29:44.460 Hail: INFO: Coerced sorted dataset 7:===(24 + 1) / 24]


FatalError: HailException: RelationalSetup.writeMetadata: file already exists: gs://pbn_pilot/proj-PBN/04_rare_SNPs/00_raw_data/05_vep_explode_coding_annot.mt

Java stack trace:
is.hail.utils.HailException: RelationalSetup.writeMetadata: file already exists: gs://pbn_pilot/proj-PBN/04_rare_SNPs/00_raw_data/05_vep_explode_coding_annot.mt
	at __C21034Compile.__m21038begin_group_0(Emit.scala)
	at __C21034Compile.__m21036split_Block(Emit.scala)
	at __C21034Compile.apply(Emit.scala)
	at is.hail.expr.ir.CompileAndEvaluate$.$anonfun$_apply$3(CompileAndEvaluate.scala:64)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at is.hail.utils.ExecutionTimer.time(ExecutionTimer.scala:98)
	at is.hail.backend.ExecuteContext.time(ExecuteContext.scala:187)
	at is.hail.expr.ir.CompileAndEvaluate$.$anonfun$_apply$2(CompileAndEvaluate.scala:64)
	at is.hail.expr.ir.CompileAndEvaluate$.$anonfun$_apply$2$adapted(CompileAndEvaluate.scala:62)
	at is.hail.backend.ExecuteContext.$anonfun$

In [22]:
# gene burden format or fisher test
mt.count()
hl.summarize_variants(mt)

Number of alleles,Count
2,144695
Allele type,Count
SNP,142129
Deletion,1625
Insertion,941
Metric,Value
Transitions,108336
Transversions,33793
Ratio,3.21
Contig,Count


In [23]:
print("the next part is to group variants by gene which falls")  # gene_symbol대신 gene_id
# reference only가 아닌 경우!
mt2 = mt.group_rows_by(mt.vep.worst_csq_by_gene_canonical.gene_id, mt.vep.worst_csq_by_gene_canonical.gene_symbol).aggregate(
#mt2 = mt.group_rows_by(mt.gene_symbol, mt.interval_from_genecode).aggregate(
    ptv = hl.agg.count_where(mt.AC5 & mt.isPTV & mt.GT.is_non_ref()),
    
    ptv_plus_mpc3 = hl.agg.count_where(mt.AC5 & (mt.isPTV | mt.mpc3) & mt.GT.is_non_ref()),
    ptv_plus_mpc2 = hl.agg.count_where(mt.AC5 & (mt.isPTV | mt.mpc2) & mt.GT.is_non_ref()),
    mpc2 = hl.agg.count_where(mt.AC5 & mt.mpc2 & mt.GT.is_non_ref()),
        
    syn = hl.agg.count_where(mt.AC5 & mt.isSYN & mt.GT.is_non_ref())
)
mt2.count() # (18371, 44906)

the next part is to group variants by gene which falls


2025-07-16 17:30:17.063 Hail: INFO: Ordering unsorted dataset with network shuffle


(17318, 70)

In [24]:
mt2 = mt2.key_rows_by(gene_symbol=mt2.gene_symbol)

In [25]:
# 1. GENCODE GTF 
gencode_ht = hl.experimental.import_gtf(
    "gs://hail-common/references/gencode/gencode.v29.annotation.gtf.bgz",
    reference_genome="GRCh38",
    skip_invalid_contigs=True)

2025-07-16 17:30:24.580 Hail: INFO: Reading table without type imputation
  Loading field 'f0' as type str (not specified)
  Loading field 'f1' as type str (not specified)
  Loading field 'f2' as type str (not specified)
  Loading field 'f3' as type int32 (user-supplied)
  Loading field 'f4' as type int32 (user-supplied)
  Loading field 'f5' as type float64 (user-supplied)
  Loading field 'f6' as type str (not specified)
  Loading field 'f7' as type int32 (user-supplied)
  Loading field 'f8' as type str (not specified)
2025-07-16 17:32:57.148 Hail: INFO: wrote table with 2742017 rows in 1 partition to /tmp/9cCYDAkWPJC1MDJW6vbSY7


In [26]:
print("annotate...")
gencode_ht = gencode_ht.key_by("gene_name")
mt2 = mt2.annotate_rows(gene_type_from_genecode=gencode_ht[mt2.row_key.gene_symbol].gene_type)
mt2 = mt2.annotate_rows(gene_id_from_genecode=gencode_ht[mt2.row_key.gene_symbol].gene_id)
mt2.count() # (18371, 44906)

annotate...


2025-07-16 17:33:18.035 Hail: INFO: Ordering unsorted dataset with network shuffle


(17318, 70)

In [27]:
print("filter to protein coding...")
mt2 = mt2.filter_rows(mt2.gene_type_from_genecode == "protein_coding") 
mt2.count() # (18334, 44906)

filter to protein coding...


2025-07-16 17:33:32.324 Hail: INFO: Ordering unsorted dataset with network shuffle
2025-07-16 17:33:37.923 Hail: INFO: Ordering unsorted dataset with network shuffle
2025-07-16 17:33:52.357 Hail: INFO: Ordering unsorted dataset with network shuffle


(17283, 70)

In [28]:
print("annotate gene's interval info...")
mt2 = mt2.annotate_rows(interval_from_genecode=gencode_ht[mt2.row_key.gene_symbol].interval)
mt2.describe()

annotate gene's interval info...
----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'gene_id': str
    'gene_symbol': str
    'gene_type_from_genecode': str
    'gene_id_from_genecode': str
    'interval_from_genecode': interval<locus<GRCh38>>
----------------------------------------
Entry fields:
    'ptv': int64
    'ptv_plus_mpc3': int64
    'ptv_plus_mpc2': int64
    'mpc2': int64
    'syn': int64
----------------------------------------
Column key: ['s']
Row key: ['gene_symbol']
----------------------------------------


In [29]:
print("write final mt2...")
mt2.write("gs://pbn_pilot/proj-PBN/04_rare_SNPs/00_raw_data/06_ac_ptv_mpc_syn.gene_group.bge_all.mt")

write final mt2...


2025-07-16 17:36:10.071 Hail: INFO: Ordering unsorted dataset with network shuffle
2025-07-16 17:37:15.230 Hail: INFO: Ordering unsorted dataset with network shuffle
2025-07-16 17:37:30.258 Hail: INFO: Ordering unsorted dataset with network shuffle
2025-07-16 17:37:42.143 Hail: INFO: Ordering unsorted dataset with network shuffle
2025-07-16 17:37:56.676 Hail: INFO: Ordering unsorted dataset with network shuffle


FatalError: HailException: RelationalSetup.writeMetadata: file already exists: gs://pbn_pilot/proj-PBN/04_rare_SNPs/00_raw_data/06_ac_ptv_mpc_syn.gene_group.bge_all.mt

Java stack trace:
is.hail.utils.HailException: RelationalSetup.writeMetadata: file already exists: gs://pbn_pilot/proj-PBN/04_rare_SNPs/00_raw_data/06_ac_ptv_mpc_syn.gene_group.bge_all.mt
	at __C39069Compile.__m39073begin_group_0(Emit.scala)
	at __C39069Compile.__m39071split_Block(Emit.scala)
	at __C39069Compile.apply(Emit.scala)
	at is.hail.expr.ir.CompileAndEvaluate$.$anonfun$_apply$3(CompileAndEvaluate.scala:64)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at is.hail.utils.ExecutionTimer.time(ExecutionTimer.scala:98)
	at is.hail.backend.ExecuteContext.time(ExecuteContext.scala:187)
	at is.hail.expr.ir.CompileAndEvaluate$.$anonfun$_apply$2(CompileAndEvaluate.scala:64)
	at is.hail.expr.ir.CompileAndEvaluate$.$anonfun$_apply$2$adapted(CompileAndEvaluate.scala:62)
	at is.hail.backend.Execut

In [31]:
mt2.show()

2025-07-16 17:39:53.711 Hail: INFO: Ordering unsorted dataset with network shuffle
2025-07-16 17:41:00.780 Hail: INFO: Ordering unsorted dataset with network shuffle
2025-07-16 17:41:15.183 Hail: INFO: Ordering unsorted dataset with network shuffle


+-------------+----------------+--------------------------+
| gene_symbol | 'PE421119'.ptv | 'PE421119'.ptv_plus_mpc3 |
+-------------+----------------+--------------------------+
| str         |          int64 |                    int64 |
+-------------+----------------+--------------------------+
| "A1BG"      |              0 |                        0 |
| "A1CF"      |              0 |                        0 |
| "A2M"       |              0 |                        0 |
| "A2ML1"     |              0 |                        0 |
| "A3GALT2"   |              0 |                        0 |
| "A4GALT"    |              0 |                        0 |
| "A4GNT"     |              0 |                        0 |
| "AAAS"      |              0 |                        0 |
| "AACS"      |              0 |                        0 |
| "AADAC"     |              0 |                        0 |
+-------------+----------------+--------------------------+

+--------------------------+-----------------+----------------+
| 'PE421119'.ptv_plus_mpc2 | 'PE421119'.mpc2 | 'PE421119'.syn |
+--------------------------+-----------------+----------------+
|                    int64 |           int64 |          int64 |
+--------------------------+-----------------+----------------+
|                        0 |               0 |              0 |
|                        0 |               0 |              0 |
|                        0 |               0 |              0 |
|                        0 |               0 |              0 |
|                        0 |               0 |              0 |
|                        0 |               0 |              0 |
|                        0 |               0 |              0 |
|                        0 |               0 |              0 |
|                        0 |               0 |              0 |
|                        0 |               0 |              0 |
+--------------------------+-----------------+----------------+
showing top 10 rows
showing the first 1 of 70 columns

In [ ]:
small_mt = mt2.key_rows_by(*mt2.row_key, 'ptv', 'ptv_plus_mpc3', 'ptv_plus_mpc2', 'mpc2', 'syn')

In [33]:
mt2.ptv_plus_mpc2.export('gs://pbn_pilot/proj-PBN/04_rare_SNPs/02_variants_annotation/PBN_ptv_plus_mpc2.gene_group.tsv')

2025-07-16 17:46:23.537 Hail: INFO: Ordering unsorted dataset with network shuffle
2025-07-16 17:47:29.499 Hail: INFO: Ordering unsorted dataset with network shuffle
2025-07-16 17:47:42.541 Hail: INFO: Ordering unsorted dataset with network shuffle
2025-07-16 17:48:54.479 Hail: INFO: merging 25 files totalling 2.4M...1) / 24]
2025-07-16 17:48:54.828 Hail: INFO: while writing:
    gs://pbn_pilot/proj-PBN/04_rare_SNPs/02_variants_annotation/PBN_ptv_plus_mpc3.gene_group.tsv
  merge time: 348.728ms


In [34]:
mt2.ptv_plus_mpc2.export('gs://pbn_pilot/proj-PBN/04_rare_SNPs/02_variants_annotation/PBN_ptv_plus_mpc2.gene_group.tsv')

2025-07-16 17:50:07.015 Hail: INFO: Ordering unsorted dataset with network shuffle
2025-07-16 17:51:12.146 Hail: INFO: Ordering unsorted dataset with network shuffle
2025-07-16 17:51:27.524 Hail: INFO: Ordering unsorted dataset with network shuffle
2025-07-16 17:52:34.312 Hail: INFO: merging 25 files totalling 2.4M...1) / 24]
2025-07-16 17:52:34.569 Hail: INFO: while writing:
    gs://pbn_pilot/proj-PBN/04_rare_SNPs/02_variants_annotation/PBN_ptv_plus_mpc2.gene_group.tsv
  merge time: 256.160ms


In [35]:
mt2.mpc2.export('gs://pbn_pilot/proj-PBN/04_rare_SNPs/02_variants_annotation/PBN_mpc2.gene_group.tsv')

2025-07-16 17:53:51.475 Hail: INFO: Ordering unsorted dataset with network shuffle
2025-07-16 17:54:58.495 Hail: INFO: Ordering unsorted dataset with network shuffle
2025-07-16 17:55:09.815 Hail: INFO: Ordering unsorted dataset with network shuffle
2025-07-16 17:56:18.388 Hail: INFO: merging 25 files totalling 2.4M...1) / 24]
2025-07-16 17:56:18.681 Hail: INFO: while writing:
    gs://pbn_pilot/proj-PBN/04_rare_SNPs/02_variants_annotation/PBN_mpc2.gene_group.tsv
  merge time: 292.226ms


In [36]:
mt2.syn.export('gs://pbn_pilot/proj-PBN/04_rare_SNPs/02_variants_annotation/PBN_syn.gene_group.tsv')

2025-07-16 17:56:34.176 Hail: INFO: Ordering unsorted dataset with network shuffle
2025-07-16 17:56:39.958 Hail: INFO: Ordering unsorted dataset with network shuffle
2025-07-16 17:56:55.085 Hail: INFO: Ordering unsorted dataset with network shuffle
2025-07-16 17:58:03.316 Hail: INFO: merging 25 files totalling 2.4M...1) / 24]
2025-07-16 17:58:03.630 Hail: INFO: while writing:
    gs://pbn_pilot/proj-PBN/04_rare_SNPs/02_variants_annotation/PBN_syn.gene_group.tsv
  merge time: 313.640ms
